

# Nettoyage de la base de données

### <u>1. Description de la base de données

**Les données** </u>

La base de données utilisée est celle de Data.gouv qui liste les festivals en France. Les critères de sélection des festivals sont les suivants :
- avoir eu lieu en 2019
- avoir connu au moins deux éditions en 2019
- se dérouler sur plus d’une journée
- compter au moins 5 spectacles, représentations, concerts ou projections
<u> 



**Description de la base de données** </u>

La base de donnée regroupe les éléments suivants :
- Identité du festival : nom, disciplines dominantes, sous-catégorie au sein des disciplines
- Données géographiques : région de déroulement, département, adresse postale, géocodage xy (coordonnées)
- Données administratives : identifiant, code insee...
- Données temporelles : période de déroulement, date de création...
- Trouver/communiquer avec le festival : site internet et adresse mail.

### <u>2. Suppression des données inutiles 
</u>

On a choisit de supprimer les colonnes suivantes : 
- Code postal (de la commune principale de déroulement)
- Code Insee commune
- Code Insee EPCI
- Libellé EPCI	
- Numéro de voie	 
- Type de voie (rue, Avenue, boulevard, etc.)	
- Nom de la voie	
- Adresse postale	
- Complément d'adresse (facultatif)
- Décennie de création du festival
- Année de création du festival
- Identifiant Agence A
- identifiant CNM

On a aussi fait le choix de ne conserver que les festivals se déroulant en France métropolitaine. 


In [ ]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('festivals_en_France.csv', sep=';', encoding='utf-8-sig')

# Afficher les premières lignes pour vérifier le contenu
print(df.head())


# Supprimer les colonnes qui ne nous intéressent pas
df = df.drop(["Code postal (de la commune principale de déroulement)", "Code Insee commune", "Code Insee EPCI", "Libellé EPCI", "Numéro de voie", "Type de voie (rue, Avenue, boulevard, etc.)", "Nom de la voie", "Adresse postale", "Complément d'adresse (facultatif)", "Décennie de création du festival", "Année de création du festival", "Identifiant Agence A", "identifiant CNM"], axis=1)

# Afficher les colonnes restantes pour vérifier
print(df.columns.tolist())


# Supprimer les festivals hors France métropolitaine
regions_non_metropolitaines = ["Guadeloupe", "Martinique", "Guyane", "La Réunion", "Mayotte", "Polynésie française", "Saint-Pierre-et-Miquelon", "Saint-Barthélemy"]
df = df[~df["Région principale de déroulement"].isin(regions_non_metropolitaines)]

# Vérifier le contenu après suppression
print(df["Région principale de déroulement"].unique())


### <u>3. Modification et mise en forme des données  

**La catégorie "pluridisciplinaire"**  </u>

Il s'agit ici d'expliciter la catégorie "pluridisciplinaire". En effet, l'utilisateur va cocher le type de festival qu'il souhaite : il faut donc spécifier à quelles disciplines appartient le festival pour que celui-ci apparaisse lorsque l'utilisateur cochera. Ainsi, si la colonne "Sous-catégorie [...]" n'est pas vide, on renomme la colonne "Discipline dominante" avec le nom de la sous-catégorie.

Par exemple : le festival "Rêves d'enfants" est noté pluridisciplinaire et est un festival de littérature et de spectacle vivant. Il faut qu'il apparaisse lorsque l'utilisateur coche "Littérature" ou "Spectacle vivant". On renomme ainsi la colonne "Discipline dominante" "Livre, littérature et Spectacle vivant". 



In [ ]:
# Liste des colonnes sous-catégories que l'on analyse
colonnes_sous_categories = [
    "Sous-catégorie spectacle vivant",
    "Sous-catégorie musique",
    "Sous-catégorie cinéma et audiovisuel",
    "Sous-catégorie arts visuels et arts numériques",
    "Sous-catégorie livre et littérature"
]

# Dictionnaire pour associer les colonnes de sous-catégorie à leurs noms dans "Discipline dominante"
mapping_discipline = {
    "Sous-catégorie spectacle vivant": "Spectacle vivant",
    "Sous-catégorie musique": "Musique",
    "Sous-catégorie cinéma et audiovisuel": "Cinéma et audiovisuel",
    "Sous-catégorie arts visuels et arts numériques": "Arts visuels et numériques",
    "Sous-catégorie livre et littérature": "Livre et littérature"
}

# Fonction pour renommer la discipline dominante en respectant les correspondances spécifiques
def renommer_discipline(row):
    # Si "Discipline dominante" est pluridisciplinaire
    if row["Discipline dominante"] == "Pluridisciplinaire":
        # Liste des disciplines à partir des sous-catégories non vides
        sous_categories = [mapping_discipline[col] for col in colonnes_sous_categories if pd.notna(row[col])]
        # Si au moins une sous-catégorie est trouvée, les utiliser
        if sous_categories:
            return " et ".join(sous_categories)
    # Sinon, garder la valeur d'origine
    return row["Discipline dominante"]

# Appliquer la fonction sur le DataFrame
df["Discipline dominante"] = df.apply(renommer_discipline, axis=1)

# Vérifier les résultats
print(df[["Nom du festival", "Discipline dominante"]].head())